<a href="https://colab.research.google.com/github/824zzy/Code_Chips/blob/master/Chinese_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

todo
### Copyright 2018 The BUPT Zhengyuan Zhu.

Licensed under the Apache License, Version 2.0 (the "License").
<table class="tfo-notebook-buttons" align="center"><td>


<td>
<a target="_blank"  href="https://github.com/BUPT/awesome-chatbot/tree/master/code"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View all sources on GitHub</a></td>
</table>
# Chinese Segmentation Based on Maximum Probability

## Prepare Data
According to slides presented by Professor Wang. It is convenient to use Beijing University's corpus which can be found [here](https://github.com/hankcs/OpenCorpus). 

All the corpus this project used are as follow:
  - dada 
  - dadad

### Download corpus from Internet

In [1]:
# download 1998 people news in January.
! wget https://github.com/hankcs/OpenCorpus/raw/master/pku98/199801.txt

--2018-11-29 03:58:45--  https://github.com/hankcs/OpenCorpus/raw/master/pku98/199801.txt
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hankcs/OpenCorpus/master/pku98/199801.txt [following]
--2018-11-29 03:58:46--  https://raw.githubusercontent.com/hankcs/OpenCorpus/master/pku98/199801.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9004042 (8.6M) [text/plain]
Saving to: ‘199801.txt’

199801.txt          100%[===================>]   8.59M  8.09MB/s    in 1.1s    

2018-11-29 03:58:47 (8.09 MB/s) - ‘199801.txt’ saved [9004042/9004042]



### Functions for data cleaning


In [21]:
def read_lines(corpus_name):
  """Read lines from different corpus
  
  # Arguments:
    corpus_name: str in corpus list
  
  # Return:
    lines: a list contains the whole corpus
  """
  with open(corpus_name, 'r') as file:
    lines = file.readlines()
  return lines


# test case for loading corpus
lines = read_lines("199801.txt")
print("Examples of corpus 1998 People News:")
print("First sentence of corpus:", lines[0])
print("Second sentence of corpus:", lines[1])

Examples of corpus 1998 People News:
19484
迈向/v	充满/v	希望/n	的/u	新/a	世纪/n	——/w	一九九八年/t	新年/t	讲话/n	（/w	附/v	图片/n	１/m	张/q	）/w

中共中央/nt	总书记/n	、/w	国家/n	主席/n	江泽民/nr



### Build dictionary: word counts dictionary and bi-gram dictionary

In [33]:
# version 1
def build_dictionary(corpus):
  """Build a dictonary based on differnet corpus for finding candidate words
  
  # Arguments:
    corpus: a list contains all sentences in corpus
    
  # Return:
    word_count_dict: a dictionary whose key represents the word and value 
                    represents the probability of its occurrence. 
  """
  word_count_dict = {}
  i = 0
  for sentence in corpus:
    print(sentence)
    word_list = [word[:-2] for word in sentence.split("\t")]
    # fix last word
    print(word_list)
    word_list[-1] = word_list[-1][:-1]
    print("hahah", word_list)
    i = i + 1
    if i > 5:
      break
    # fix tag more than one char
    for i, word in enumerate(word_list):
      for j, char in enumerate(word):
        if char == '/':
          word_list[i] = word[:j]
          if word[0] == '[':
            word_list[i] = word_list[i][1:]
          break
        else:
          if word[0] == '[':
            word_list[i] = word_list[i][1:]
          
    # build dictionary via sentence
    for word in word_list:
      if word_count_dict.get(word) != None:
        word_count_dict[word] += 1
      else:
        word_count_dict[word] = 1
        
  return word_count_dict

# build dictonary using people news
word_count_dict = build_dictionary(lines)
# calculate probability of each items
# for k, v in word_prob_dict.items():
#   word_prob_dict[k] /= len(word_prob_dict)
print(word_count_dict)  

迈向/v	充满/v	希望/n	的/u	新/a	世纪/n	——/w	一九九八年/t	新年/t	讲话/n	（/w	附/v	图片/n	１/m	张/q	）/w

['迈向', '充满', '希望', '的', '新', '世纪', '——', '一九九八年', '新年', '讲话', '（', '附', '图片', '１', '张', '）/']
hahah ['迈向', '充满', '希望', '的', '新', '世纪', '——', '一九九八年', '新年', '讲话', '（', '附', '图片', '１', '张', '）']
中共中央/nt	总书记/n	、/w	国家/n	主席/n	江泽民/nr

['中共中央/', '总书记', '、', '国家', '主席', '江泽民/n']
hahah ['中共中央/', '总书记', '、', '国家', '主席', '江泽民/']
{'迈向': 1, '充满': 1, '希望': 1, '的': 1, '新': 1, '世纪': 1, '——': 1, '一九九八年': 1, '新年': 1, '讲话': 1, '（': 1, '附': 1, '图片': 1, '１': 1, '张': 1, '）': 1}


In [133]:
# version2 is better!
def build_dictionary(corpus):
  """Build a dictonary based on differnet corpus for finding candidate words
  
  # Arguments:
    corpus: a list contains all sentences in corpus
    
  # Return:
    word_count_dict: a dictionary whose key represents the word and value 
                    represents the count of its occurrence. 
  """
  word_count_dict = {}
  i = 0
  for index, sentence in enumerate(corpus):
    if index == 0:
      print("Example of sentence in corpus: ", sentence)
    # seperate word by tab
    word_list = [word for word in sentence.split("\t")]
    if index == 0:
      print("seperate word by tab: ", word_list)
    # drop tagging in word
    for i, word in enumerate(word_list):
      word_list[i] = word.split('/')[0]
      if index == 0:
        print("splited by /:", word)     
    # build dictionary via sentence
    for word in word_list:
      if word_count_dict.get(word) != None:
        word_count_dict[word] += 1
      else:
        word_count_dict[word] = 1
        
  return word_count_dict

# build dictonary using people news
word_count_dict = build_dictionary(lines)
print("Finally we obtain the dictonary: ", word_count_dict)  

Example of sentence in corpus:  迈向/v	充满/v	希望/n	的/u	新/a	世纪/n	——/w	一九九八年/t	新年/t	讲话/n	（/w	附/v	图片/n	１/m	张/q	）/w

seperate word by tab:  ['迈向/v', '充满/v', '希望/n', '的/u', '新/a', '世纪/n', '——/w', '一九九八年/t', '新年/t', '讲话/n', '（/w', '附/v', '图片/n', '１/m', '张/q', '）/w\n']
splited by /: 迈向/v
splited by /: 充满/v
splited by /: 希望/n
splited by /: 的/u
splited by /: 新/a
splited by /: 世纪/n
splited by /: ——/w
splited by /: 一九九八年/t
splited by /: 新年/t
splited by /: 讲话/n
splited by /: （/w
splited by /: 附/v
splited by /: 图片/n
splited by /: １/m
splited by /: 张/q
splited by /: ）/w

Finally we obtain the dictonary:  {'迈向': 33, '充满': 115, '希望': 506, '的': 54487, '新': 2253, '世纪': 441, '——': 157, '一九九八年': 39, '新年': 274, '讲话': 233, '（': 4316, '附': 294, '图片': 623, '１': 599, '张': 452, '）': 4316, '中共中央': 67, '总书记': 97, '、': 23116, '国家': 1722, '主席': 638, '江泽民': 446, '一九九七年': 58, '十二月': 26, '三十一日': 11, '１２月': 246, '３１日': 57, '，': 74921, '发表': 301, '１９９８年': 315, '《': 1937, '》': 1940, '。': 35983, '新华社': 1170, '记者': 2143, '兰红光'

In [67]:
def bigram_dictionary(corpus):
  """Build Bigram dictionary by corpus
  The key is a bigram, and The value is a 
    Ex: d
  
  """
  bigram_dict = {}
  for epi, sentence in enumerate(corpus):
    # seperate word by tab
    word_list = [word for word in sentence.split("\t")]
    # drop tagging in word
    for i, word in enumerate(word_list):
      word_list[i] = word.split('/')[0]
    # build bigram count for dictionary
    for pos, word in enumerate(word_list):
      if pos < len(word_list) - 1:
        bigram = word_list[pos] + ' ' + word_list[pos + 1]
        if bigram_dict.get(bigram) != None:
          bigram_dict[bigram] += 1
        else:
          bigram_dict[bigram] = 1
    if epi == 0:
      print("Example of bigram dictionary in first sentence:\n", bigram_dict)
  return bigram_dict
          
bigram_dict = bigram_dictionary(lines)
print("We have ", len(bigram_dict), " bigram pairs!!")


Example of bigram dictionary in first sentence:
 {'迈向 充满': 1, '充满 希望': 1, '希望 的': 1, '的 新': 1, '新 世纪': 1, '世纪 ——': 1, '—— 一九九八年': 1, '一九九八年 新年': 1, '新年 讲话': 1, '讲话 （': 1, '（ 附': 1, '附 图片': 1, '图片 １': 1, '１ 张': 1, '张 ）': 1}
We have  458507  bigram pairs!!


## Baseline: N-gram model

In [0]:
class BiGram(object):
  """Apply N-Gram model to Bigram.
  
  A bigram or digram is a sequence of two adjacent elements from a string of 
  tokens, which are typically letters, syllables, or words. 
  A bigram is an n-gram for n=2. The frequency distribution of every bigram in 
  a string is commonly used for simple statistical analysis of text in many 
  applications, including in computational linguistics, cryptography, speech 
  recognition, and so on.
  
  # Arguments:
    corpus:
    word_count_dict:
    bigram_dict: 
    max_split: a int numrepresenting the sliding window's size
    word_num: a int num representing length of dictionary's words
    
    
  
  # References:
    - [Bigram](https://en.wikipedia.org/wiki/Bigram)
  
  """
  
  
  def __init__(self, corpus, word_count_dict, bigram_dict, max_split=4):
    self.corpus = corpus
    self.word_count_dict = word_count_dict
    self.max_split = max_split
    self.bigram_dict = bigram_dict
    self.word_num = len(word_count_dict)

  def candidate_words(self, sentence):
    """scan the whole sentence from left to right, we will obtain candidate 
    words for determining left-neighbor words.

    # Arguments:
      sentence: str, content of corpus

    # Return:
      candidate_words: a list of candidate words.
    """
    candidate_words = []
    # search the whole sentence
    for start_pos, word in enumerate(sentence):
      candidate_words.append([word, start_pos, start_pos])
      # check out the range of object sentence
      for max_pos in range(1, self.max_split):
        current_pos = start_pos + max_pos
        if current_pos < len(sentence):
          word = word + sentence[current_pos]
          if word in self.word_count_dict.keys():
            candidate_words.append([word, start_pos, current_pos])
    return candidate_words
  
  def probable_segments(self, sentence):
    candidate_words = self.candidate_words(sentence)
#     count = 0
    for word in candidate_words:
#       # prevent for long time parsing
#       if count > 1000:
#         break
      if word[1] == 0 and word[2] != len(sentence) - 1:
#         # ???
#         no_whitespace_sen = ''.join(word[0].split())
        for word in candidate_words:
          if word[1] == 0 and word[2] != len(sentence) - 1:
            end = word[2]
            for later_word in candidate_words:
              if later_word[1] == end + 1:  # 如果later_word是当前词的后续词，那么拼接到当前词上
                word_seq = [word[0] + ' ' + later_word[0], word[1], later_word[2]]  # 合并
                candidate_words.append(word_seq)
                # print('拼出了新词：%s' % word_seq)
#                 count += 1
            candidate_words.remove(word)  # 遍历完后，这个开头部分短语要移除掉，不然下次遍历还会对它做无用功
    print('所有结果词序列有：%s' % candidate_words)

    word_segment_res_list = []  # 存储分词结果序列
    for seque in candidate_words:
        if seque[1] == 0 and seque[2] == len(sentence) - 1:
            word_segment_res_list.append(seque[0])
    print('获得的所有分词结果是：')
    print(word_segment_res_list)
    return word_segment_res_list
  
  def sen_log_prob(self, sequence, display=False):
    """ 
    """
    word_list = sequence.split(' ')
    prob_total = 0.0
    word_start = word_list[0]
    # first word prob with laplasian smooth
    if word_start in self.word_count_dict.keys():
      count = self.word_count_dict[word_start]
    else:
      count = 1

    prob_total += math.log(count / self.word_num)
    # calculate Bigram Probability
    for i in range(len(word_list) - 1):
      prev_w = word_list[i]
      later_w = word_list[i+1]
      bigram = prev_w + " " + later_w
      if bigram in self.bigram_dict.keys():
        count = self.bigram_dict[bigram]
      else:
        count = 1
      prob_total += math.log(count / self.word_num)
    
    prob = math.pow(math.e, prob_total)
    if display:
      print('The probability of ', sequence, " is ", prob)
    
    return prob
  
  def maximum_probability(self, sequence_list, display=False):
    max_prob = 0.0
    seg_result = ""
    for seq in sequence_list:
      prob = self.sen_log_prob(seq)
      if max_prob < prob:
        max_prob, seg_result = prob, seq 
    if display:
      print('最好的分词结果（概率为%s）是 ：%s' % (max_prob, seg_result))
    return seg_result

  def segment_testset(self, test_set):
    """
    """
    corpus_result = ''
    for sentence in test_set:
      prob_seqs = self.probable_segments(sentence)
      seg_result = self.maximum_probability(prob_seqs, display=True)
        
      
      
      
  
  
      
      


        
    
            
            


  

### Test case for N-Gram model

In [131]:
import math
# test cases
## test candidate_words
bigram = BiGram(lines, word_prob_dict, bigram_dict, 4)
cand_w = bigram.candidate_words("太阳当空照，江泽民对我笑")
print("candidate_word test case:", cand_w)
print("\n"*2 + "-"*50 + "\n"*2)
## test segment
seg_result = bigram.probable_segments("太阳当空照，江泽民对我笑")
print("\n"*2 + "-"*50 + "\n"*2)

## test computation of sentence probability
for each_seg in seg_result:
  bigram.sen_log_prob(each_seg, display=True)
print("\n"*2 + "-"*50 + "\n"*2)
  
# test maximum_probability
bigram.maximum_probability(seg_result, display=True)
print("\n"*2 + "-"*50 + "\n"*2)

# test segment_testset function
bigram.segment_testset(["PFR人民日报标注语料库(版本1.0，下面简称PFR语料库)", "是在得到人民日报社新闻信息中心许可的条件下", "以1998年人民日报语料为对象", "由北京大学计算语言学研究所和富士通研究开发中心有限公司共同制作的标注语料库。", "该语料库对600多万字节的中文文章进行了分词及词性标注", "其被作为原始数据应用于大量的研究和论文中。"])
print("\n"*2 + "-"*50 + "\n"*2)

candidate_word test case: [['太', 0, 0], ['太阳', 0, 1], ['阳', 1, 1], ['当', 2, 2], ['当空', 2, 3], ['空', 3, 3], ['照', 4, 4], ['，', 5, 5], ['江', 6, 6], ['江泽民', 6, 8], ['泽', 7, 7], ['泽民', 7, 8], ['民', 8, 8], ['对', 9, 9], ['我', 10, 10], ['笑', 11, 11]]


--------------------------------------------------


所有结果词序列有：[['阳', 1, 1], ['当', 2, 2], ['当空', 2, 3], ['空', 3, 3], ['照', 4, 4], ['，', 5, 5], ['江', 6, 6], ['江泽民', 6, 8], ['泽', 7, 7], ['泽民', 7, 8], ['民', 8, 8], ['对', 9, 9], ['我', 10, 10], ['笑', 11, 11], ['太 阳 当空 照 ， 江 泽民 对 我 笑', 0, 11], ['太 阳 当空 照 ， 江 泽 民 对 我 笑', 0, 11], ['太阳 当空 照 ， 江泽民 对 我 笑', 0, 11], ['太 阳 当空 照 ， 江泽民 对 我 笑', 0, 11], ['太阳 当 空 照 ， 江 泽民', 0, 8], ['太 阳 当 空 照 ， 江泽民 对 我 笑', 0, 11], ['太 阳 当 空 照 ， 江 泽民 对 我 笑', 0, 11], ['太 阳 当 空 照 ， 江 泽 民 对 我 笑', 0, 11], ['太阳 当空 照 ， 江 泽 民 对 我 笑', 0, 11], ['太阳 当 空 照 ， 江 泽 民 对 我 笑', 0, 11], ['太阳 当 空 照 ， 江泽民 对 我 笑', 0, 11], ['太阳 当空 照 ， 江 泽民 对 我 笑', 0, 11]]
获得的所有分词结果是：
['太 阳 当空 照 ， 江 泽民 对 我 笑', '太 阳 当空 照 ， 江 泽 民 对 我 笑', '太阳 当空 照 ， 江泽民 对 我 笑', '太 阳 当空 照 ， 江

KeyboardInterrupt: ignored

## Measure the performence of N-Gram Model

In [105]:
test_set = []
with open("test.txt", 'r') as f:
  for line in f.readlines():
    test_set.append(line)

Bigram_model = BiGram(lines, word_count_dict, bigram_dict, max_split=4, test_set = test_set)
result = Bigram_model.maximum_probability()

    

AttributeError: ignored